# question 1

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load MNIST data

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28*28).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28*28).astype('float32') / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 81s 7us/step


# Build the MLP model

In [3]:
def create_model(hidden_units=128, dropout_rate=0.2):
    model = Sequential([
        Dense(hidden_units, activation='relu', input_shape=(28*28,)),
        Dropout(dropout_rate),
        Dense(hidden_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(10, activation='softmax')
    ])
    return model

# Hyperparameter tuning

In [4]:
hidden_units_list = [128, 256, 512]
dropout_rates = [0.2, 0.3, 0.5]
best_accuracy = 0
best_params = {}

In [5]:
for hidden_units in hidden_units_list:
    for dropout_rate in dropout_rates:
        model = create_model(hidden_units=hidden_units, dropout_rate=dropout_rate)
        model.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        print(f"Training model with {hidden_units=} and {dropout_rate=}")
        history = model.fit(x_train, y_train, 
                            validation_data=(x_test, y_test),
                            epochs=10, batch_size=128, verbose=0)
        accuracy = model.evaluate(x_test, y_test, verbose=0)[1]
        print(f"Accuracy: {accuracy}")
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {'hidden_units': hidden_units, 'dropout_rate': dropout_rate}

print(f"Best accuracy: {best_accuracy} with {best_params=}")


c:\Users\pouya\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training model with hidden_units=128 and dropout_rate=0.2
Accuracy: 0.9786999821662903
Training model with hidden_units=128 and dropout_rate=0.3
Accuracy: 0.9803000092506409
Training model with hidden_units=128 and dropout_rate=0.5
Accuracy: 0.9731000065803528
Training model with hidden_units=256 and dropout_rate=0.2
Accuracy: 0.9821000099182129
Training model with hidden_units=256 and dropout_rate=0.3
Accuracy: 0.9799000024795532
Training model with hidden_units=256 and dropout_rate=0.5
Accuracy: 0.9799000024795532
Training model with hidden_units=512 and dropout_rate=0.2
Accuracy: 0.9800999760627747
Training model with hidden_units=512 and dropout_rate=0.3
Accuracy: 0.9829999804496765
Training model with hidden_units=512 and dropout_rate=0.5
Accuracy: 0.9825999736785889
Best accuracy: 0.9829999804496765 with best_params={'hidden_units': 512, 'dropout_rate': 0.3}


# Train final model with best parameters

In [6]:
final_model = create_model(**best_params)
final_model.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
final_model.fit(x_train, y_train, 
                validation_data=(x_test, y_test),
                epochs=20, batch_size=128, verbose=1)

Epoch 1/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8506 - loss: 0.4848 - val_accuracy: 0.9657 - val_loss: 0.1062
Epoch 2/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9647 - loss: 0.1154 - val_accuracy: 0.9737 - val_loss: 0.0807
Epoch 3/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9742 - loss: 0.0825 - val_accuracy: 0.9750 - val_loss: 0.0791
Epoch 4/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9783 - loss: 0.0688 - val_accuracy: 0.9805 - val_loss: 0.0666
Epoch 5/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9823 - loss: 0.0545 - val_accuracy: 0.9794 - val_loss: 0.0688
Epoch 6/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9840 - loss: 0.0494 - val_accuracy: 0.9812 - val_loss: 0.0662
Epoch 7/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9858 - loss: 0.0441 - val_accuracy: 0.9810 - val_loss: 0.0636
Epoch 8/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9874 - loss: 0.0388 - val_accuracy: 0.

# question 2

In [19]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

# Load dataset

In [20]:
data = load_breast_cancer()
X = data.data
y = data.target

# Train-test split

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale the data

In [22]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the model

In [23]:
def create_model(neurons_1=64, neurons_2=32, dropout_rate=0.3, learning_rate=0.001):
    model = Sequential([
        Dense(neurons_1, activation='relu', input_dim=X_train.shape[1]),
        Dropout(dropout_rate),
        Dense(neurons_2, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Wrap the model for GridSearch


In [24]:
model = KerasClassifier(model=create_model, verbose=0)

# Define hyperparameters to search

In [25]:
param_grid = {
    'model__neurons_1': [32, 64, 128],
    'model__neurons_2': [16, 32, 64],
    'model__dropout_rate': [0.2, 0.3, 0.5],
    'model__learning_rate': [0.001, 0.01],
    'batch_size': [16, 32],
    'epochs': [50, 100]
}

# Perform Grid Search

In [26]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=1, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits


c:\Users\pouya\anaconda3\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
c:\Users\pouya\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
c:\Users\pouya\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Output best parameters and results


In [27]:
print("Best Parameters:", grid_result.best_params_)
print("Best Score:", grid_result.best_score_)

Best Parameters: {'batch_size': 16, 'epochs': 50, 'model__dropout_rate': 0.5, 'model__learning_rate': 0.001, 'model__neurons_1': 32, 'model__neurons_2': 64}
Best Score: 0.9846491228070176


# Train the best model on the full training set

In [28]:
best_model = grid_result.best_estimator_
best_model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=1)

Epoch 1/100


c:\Users\pouya\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6311 - loss: 0.6478   
Epoch 2/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8699 - loss: 0.3740 
Epoch 3/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9072 - loss: 0.2782 
Epoch 4/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9299 - loss: 0.2147 
Epoch 5/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9472 - loss: 0.1681 
Epoch 6/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9605 - loss: 0.1392 
Epoch 7/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9577 - loss: 0.1445 
Epoch 8/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9705 - loss: 0.1052 
Epoch 9/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9566 - loss: 0.1217 
Epoch 10/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9589 - loss: 0.1348 
Epoch 11/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9769 - loss: 0.1068 
Epoch 12/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy

KerasClassifier(
	model=<function create_model at 0x000002DD77BA3920>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=16
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=50
	class_weight=None
	model__dropout_rate=0.5
	model__learning_rate=0.001
	model__neurons_1=32
	model__neurons_2=64
)

# Evaluate the best model

In [29]:
y_pred = (best_model.predict(X_test) > 0.5).astype(int).flatten()
print("Test Accuracy:", accuracy_score(y_test, y_pred))

Test Accuracy: 0.956140350877193
